In [1]:
# import문 보고 필요한 pip들 수동으로 설치하기 (requirements 하면 오류 많이 날수도)
# torch 최신버전이면 다 될듯!
!pwd
# 폴더 만들어졌으면 구글 드라이브에서 다운로드 
# model-v3 사용하면 됨 (임시로 epoch30으로 학습된 것)
# 구글 드라이브 url : https://drive.google.com/drive/folders/1rmU70p2WEGA0sLhsw9WYowXrTj1y7M_g?usp=sharing

# checkpoint 폴더 -> 모델 저장 (model-v3)
# data 폴더 -> txt 파일들 저장(category, answer_v2 / input_v2는 학습할 때 사용해서 다운 안받아도 됨)

/workspace/kobert-wellness-chatbot


In [2]:
%cd /workspace/kobert-wellness-chatbot

/workspace/kobert-wellness-chatbot


In [3]:
import torch
import torch.nn as nn
import random
import os
import csv
import logging
from torch.utils.data import dataloader

# KoElectra 
from model.dataloader_electra import WellnessTextClassificationDataset
from model.classifier_electra import KoELECTRAforSequenceClassfication
from transformers import ElectraModel, ElectraConfig, ElectraTokenizer

# from transformers import AutoTokenizer


# warning 출력 안되게
logging.getLogger("transformers").setLevel(logging.ERROR)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
# data 경로 지정
root_path = "."
data_path = f"{root_path}/data/input_v4.txt"
category_path = f"{root_path}/data/category.txt"
answer_path = f"{root_path}/data/answer_v2.txt"
checkpoint_path = f"{root_path}/checkpoint/model_electra_v2.pth"

In [5]:
def load_wellness_answer():

    c_f = open(category_path, 'r')
    a_f = open(answer_path, 'r')

    category_lines = c_f.readlines()
    answer_lines = a_f.readlines()

    category = {}
    answer = {}
    for line_num, line_data in enumerate(category_lines):
        data = line_data.split('    ')
        if len(data) != 2:
            print(f"Error in category file at line {line_num}: {line_data}")
        category[data[1][:-1]] = data[0]

    for line_num, line_data in enumerate(answer_lines):
        data = line_data.split('    ')
        keys = answer.keys()
        if len(data) != 2:
            print(f"Error in answer file at line {line_num}: {line_data}")
        if (data[0] in keys):
            answer[data[0]] += [data[1][:-1]]
        else:
            answer[data[0]] = [data[1][:-1]]
    return category, answer


def load_model(checkpoint_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_config = ElectraConfig.from_pretrained("monologg/koelectra-base-v3-generator")
    model = KoELECTRAforSequenceClassfication(model_config, num_labels=432, hidden_dropout_prob=0.1)    
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-generator")
    return model, tokenizer, device

def preprocess_input(tokenizer, sent, device, max_seq_len=512):
    index_of_words = tokenizer.encode(sent)
    token_type_ids = [0] * len(index_of_words)
    attention_mask = [1] * len(index_of_words)
    padding_length = max_seq_len - len(index_of_words)
    index_of_words += [0] * padding_length
    token_type_ids += [0] * padding_length
    attention_mask += [0] * padding_length

    data = {
        'input_ids': torch.tensor([index_of_words]).to(device),
        'token_type_ids': torch.tensor([token_type_ids]).to(device),
        'attention_mask': torch.tensor([attention_mask]).to(device),
        }
    return data

In [6]:
# !pip install sentence-transformers

In [7]:
def get_answer(category, answer, output, input_sentence): # 수정
    softmax_logit = torch.softmax(output[0], dim=-1).squeeze()
    max_index = torch.argmax(softmax_logit).item()
    max_index_value = softmax_logit[torch.argmax(softmax_logit)].item()

    ####### threshold 값 조절 #######
    threshold = 0.3  
    ####### threshold 값 조절 #######

    selected_categories = []
    for i, value in enumerate(softmax_logit):
        if value > threshold:
            if str(i) in category:
                selected_categories.append(category[str(i)])
                print(f"Softmax 값이 threshold({threshold}) 이상인 카테고리: {category[str(i)]}")

    # 선택된 카테고리가 없을 경우 예외처리
    if not selected_categories:
        return "선택된 카테고리가 없습니다. 다시 입력해주세요", None, max_index_value, []
    
    # 선택된 카테고리들에 해당하는 답변을 리스트에 저장
    all_answers = []
    for category_name in selected_categories:
        if category_name in answer:
            all_answers.extend(answer[category_name])

    # 만약 답변이 없다면 예외처리
    if not all_answers:
        return "선택된 카테고리에 대한 답변이 없습니다.", None, max_index_value, []

    # 답변들 중에서 랜덤으로 선택된 답변 출력
    selected_answer = random.choice(all_answers)
    # print("선택된 카테고리에 해당하는 모든 답변:")
    # print(all_answers)


    return selected_answer, selected_categories, max_index_value, all_answers

In [8]:
from kobert_transformers import get_kobert_model

def find_most_similar_sentence(input_sentence, candidate_sentences, output):
    model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-generator")
    # tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
    tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-generator")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    def get_sentence_embedding(sentence):
        inputs = tokenizer.encode_plus(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        sentence_embedding = outputs.last_hidden_state.mean(dim=1)
        return sentence_embedding

    # 입력 문장 임베딩 구하기
    input_embedding = get_sentence_embedding(input_sentence)

    # 후보 문장들과의 유사도 계산
    max_similarity = -1
    most_similar_sentence = None
    for candidate in candidate_sentences:
        candidate_embedding = get_sentence_embedding(candidate)
        similarity = torch.cosine_similarity(input_embedding, candidate_embedding, dim=1)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_sentence = candidate

    return most_similar_sentence

In [9]:
def chatbot_main(checkpoint_path, category, answer):
    model, tokenizer, device = load_model(checkpoint_path)

    sent1 = input('\nQuestion: ')
    sent = str(sent1)
    if '안녕?' in sent or '안녕!' in sent or '안녕' in sent:
            print('Answer : 반가워요! 저는 기룡이에요!')
            
    data = preprocess_input(tokenizer, sent, device, 512)
    output = model(**data)
    answer, category, max_index_value, all_answers = get_answer(category, answer, output, sent)
    # 유사한 문장 찾기
    most_similar_sentence = find_most_similar_sentence(sent, all_answers, output)
    print(f'index: {category} \nsoftmax_value: {max_index_value} \nAnswer: {answer} \n가장 유사한 문장: {most_similar_sentence} ')
    print('-' * 70)

# 카테고리 / 답변 로드        
category, answer = load_wellness_answer()

In [ ]:
while 1 :
    chatbot_main(checkpoint_path, category, answer)